In [2]:
spot = 2.45
strike = 2.5
maturity = 0.25
r = 0.05
vol = 0.25

In [3]:
#基于BS公式的期权定价
from math import log, sqrt, exp
from scipy.stats import norm

def call_option_pricer(spot, strike, maturity, r, vol):

    d1 = (log(spot/strike) + (r + 0.5 * vol *vol) * maturity) / vol / sqrt(maturity)
    d2 = d1 - vol * sqrt(maturity)

    price = spot * norm.cdf(d1) - strike * exp(-r*maturity) * norm.cdf(d2)
    return price

In [4]:
print('期权价格是：{0:.5f}'.format(call_option_pricer(spot, strike, maturity, r, vol)))

期权价格是：0.11333


使用numpy实现批量计算

普通循环计算

In [5]:
import time
import numpy as np

portfolioSize = range(1, 10000, 500)
timeSpent = []

for size in portfolioSize:
    now = time.time()
    strikes = np.linspace(2.0,3.0,size)
    for i in range(size):
        res = call_option_pricer(spot, strikes[i], maturity, r, vol)
    timeSpent.append(time.time() - now)

In [6]:
%matplotlib
from matplotlib import pylab
import seaborn as sns

sns.set(style="ticks")
pylab.figure(figsize = (12,8))
pylab.bar(portfolioSize, timeSpent, color = 'r', width =300)
pylab.grid(True)
pylab.title(u'期权计算时间耗时（单位：秒）',  fontsize = 18)
pylab.ylabel(u'时间（s)',  fontsize = 15)
pylab.xlabel(u'组合数量',  fontsize = 15)


Using matplotlib backend: Qt5Agg


使用numpy向量计算

In [7]:
sample = np.linspace(1.0, 100.0, 5)
np.exp(sample)

array([  2.71828183e+00,   1.52434373e+11,   8.54813429e+21,
         4.79357761e+32,   2.68811714e+43])

In [8]:
# 使用numpy的向量函数重写Black - Scholes公式
def call_option_pricer_nunmpy(spot, strike, maturity, r, vol):

    d1 = (np.log(spot/strike) + (r + 0.5 * vol *vol) * maturity) / vol / np.sqrt(maturity)
    d2 = d1 - vol * np.sqrt(maturity)

    price = spot * norm.cdf(d1) - strike * np.exp(-r*maturity) * norm.cdf(d2)
    return price

In [9]:
timeSpentNumpy = []
for size in portfolioSize:
    now = time.time()
    strikes = np.linspace(2.0,3.0, size)
    res = call_option_pricer_nunmpy(spot, strikes, maturity, r, vol)
    timeSpentNumpy.append(time.time() - now)

In [10]:
pylab.figure(figsize = (12,8))
pylab.bar(portfolioSize, timeSpentNumpy, color = 'r', width = 300)
pylab.grid(True)
pylab.title(u'期权计算时间耗时（单位：秒）- numpy加速版',  fontsize = 18)
pylab.ylabel(u'时间（s)',fontsize = 15)
pylab.xlabel(u'组合数量',  fontsize = 15)


In [11]:
fig = pylab.figure(figsize = (12,8))
ax = fig.gca()
pylab.plot(portfolioSize, np.log10(timeSpent), portfolioSize, np.log(timeSpentNumpy))
pylab.grid(True)
from matplotlib.ticker import FuncFormatter
def millions(x, pos):
    'The two args are the value and tick position'
    return '$10^{%.0f}$' % (x)
formatter = FuncFormatter(millions)
ax.yaxis.set_major_formatter(formatter)
pylab.title(u'期权计算时间耗时（单位：秒）',  fontsize = 18)
pylab.legend([u'循环计算', u'numpy向量加速'], loc = 'upper center', ncol = 2)
pylab.ylabel(u'时间（秒)',  fontsize = 15)
pylab.xlabel(u'组合数量',  fontsize = 15)


C:\local\Anaconda3-4.3.1-Windows-x86_64\lib\site-packages\ipykernel\__main__.py:3: RuntimeWarning: divide by zero encountered in log
  app.launch_new_instance()


使用scipy做模拟计算

In [12]:
import scipy
scipy.random.randn(10)

array([-1.55651354, -0.38803349, -1.86371403, -2.09241343, -0.00991801,
       -0.36614416,  0.48052448,  0.00959094, -1.52584825, -1.72292487])

In [13]:
pylab.figure(figsize = (12, 8))
randomSeries = scipy.random.randn(10000)
pylab.plot(randomSeries)
print('均值',randomSeries.mean())
print('标准值',randomSeries.std())

均值 0.0123920137687
标准值 0.98543805243


In [14]:
# 期权计算的蒙特卡洛方法
def call_option_pricer_monte_carlo(spot, strike, maturity, r, vol, numOfPath = 5000):
    randomSeries = scipy.random.randn(numOfPath)
    s_t = spot * np.exp((r - 0.5 * vol * vol) * maturity + randomSeries * vol * sqrt(maturity))
    sumValue = np.maximum(s_t - strike, 0.0).sum()
    price = exp(-r*maturity) * sumValue / numOfPath
    return price

In [15]:
call_option_pricer_monte_carlo(spot, strike, maturity, r, vol)

0.11086162956661633

In [16]:
res

array([ 0.47958486,  0.47948593,  0.47938701, ...,  0.00942269,
        0.00941679,  0.00941089])

计算隐含波动率

In [18]:
# 目标函数，目标价格由target确定
class cost_function:
    def __init__(self, target):
        self.targetValue = target

    def __call__(self, x):
        return call_option_pricer(spot, strike, maturity, r, x) - self.targetValue

In [21]:
# 假设我们使用vol初值作为目标
target = call_option_pricer(spot, strike, maturity, r, vol)
cost_sampel = cost_function(target)

# 使用Brent算法求解
impliedVol = scipy.optimize.brentq(cost_sampel, 0.001, 0.5)

In [23]:
print('imvol.{0:.3f}'.format(impliedVol))

imvol.0.250
